In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Trellathon/clean data.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.drop(['Unnamed: 0','User_Interest19','User_Interest38','User_Interest58','User_Interest84','User_Interest105','User_Interest107', 'User_Interest156','User_Interest157'],axis = 1, inplace = True)
df

,userId,trailId,Category_4,Category_2,Category_5,Category_47,Category_10,Category_22,Category_19,Category_14,Category_31,Category_16,Category_32,Category_9,Category_6,Category_20,Category_11,Category_18,languageId19,languageId38,languageId58,languageId84,languageId105,languageId107,languageId156,languageId157,event_type
0,57753482,5498041,1,1,1,0.0,0.0,0.0,1,0.0,1,1,0.0,1,1,0,0.0,1,0,0,1,0,0,0,0,0,like
1,48214058,2908808,1,0,1,0.0,0.0,0.0,1,0.0,0,0,0.0,1,1,0,0.0,1,0,1,0,0,0,0,0,0,comment
2,37681988,2260711,1,1,1,0.0,0.0,1.0,1,1.0,1,1,1.0,1,1,1,0.0,1,0,1,0,0,0,0,0,0,comment
3,42186799,8522388,0,0,0,0.0,0.0,0.0,1,0.0,0,0,0.0,1,1,0,0.0,1,0,0,0,1,0,0,0,0,like
4,94838388,8537147,1,1,1,0.0,0.0,0.0,1,0.0,1,0,0.0,1,0,0,0.0,1,0,0,1,0,0,0,0,0,like
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,32503582,2887903,1,1,1,0.0,0.0,0.0,1,0.0,0,0,0.0,1,1,0,0.0,1,0,1,0,0,0,0,0,0,comment
499996,25960806,3197089,1,1,1,0.0,0.0,0.0,1,0.0,1,0,0.0,1,1,0,0.0,1,0,0,1,0,0,0,0,0,save
499997,1872777,2316799,1,1,1,0.0,0.0,1.0,1,0.0,0,0,0.0,1,1,1,0.0,1,0,0,1,0,0,0,0,0,comment
499998,2116446,2319621,1,1,1,0.0,0.0,0.0,0,0.0,0,0,0.0,1,1,1,0.0,1,0,1,0,0,0,0,0,0,comment


In [ ]:
df["Category_47"].fillna(0, inplace = True) 
df["Category_10"].fillna(0, inplace = True) 
df["Category_22"].fillna(0, inplace = True) 
df["Category_32"].fillna(0, inplace = True) 
df["Category_11"].fillna(0, inplace = True) 
df["Category_14"].fillna(0, inplace = True) 
df.isnull().sum()

userId           0
trailId          0
Category_4       0
Category_2       0
Category_5       0
Category_47      0
Category_10      0
Category_22      0
Category_19      0
Category_14      0
Category_31      0
Category_16      0
Category_32      0
Category_9       0
Category_6       0
Category_20      0
Category_11      0
Category_18      0
languageId19     0
languageId38     0
languageId58     0
languageId84     0
languageId105    0
languageId107    0
languageId156    0
languageId157    0
event_type       0
dtype: int64

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df['event_type'] = labelencoder.fit_transform(df['event_type'])


In [ ]:
table_1 = df[['userId', 'trailId','event_type']]
table_2 = df[['trailId','Category_4','Category_2' ,'Category_5','Category_47','Category_10','Category_22','Category_19','Category_14','Category_31','Category_16','Category_32','Category_9','Category_6','Category_20','Category_11','Category_18','languageId19','languageId38','languageId58','languageId84','languageId105', 'languageId107','languageId156','languageId157']]



In [ ]:
table_1

,userId,trailId,event_type
0,57753482,5498041,1
1,48214058,2908808,0
2,37681988,2260711,0
3,42186799,8522388,1
4,94838388,8537147,1
...,...,...,...
499995,32503582,2887903,0
499996,25960806,3197089,2
499997,1872777,2316799,0
499998,2116446,2319621,0


In [ ]:
table_2

,trailId,Category_4,Category_2,Category_5,Category_47,Category_10,Category_22,Category_19,Category_14,Category_31,Category_16,Category_32,Category_9,Category_6,Category_20,Category_11,Category_18,languageId19,languageId38,languageId58,languageId84,languageId105,languageId107,languageId156,languageId157
0,5498041,1,1,1,0.0,0.0,0.0,1,0.0,1,1,0.0,1,1,0,0.0,1,0,0,1,0,0,0,0,0
1,2908808,1,0,1,0.0,0.0,0.0,1,0.0,0,0,0.0,1,1,0,0.0,1,0,1,0,0,0,0,0,0
2,2260711,1,1,1,0.0,0.0,1.0,1,1.0,1,1,1.0,1,1,1,0.0,1,0,1,0,0,0,0,0,0
3,8522388,0,0,0,0.0,0.0,0.0,1,0.0,0,0,0.0,1,1,0,0.0,1,0,0,0,1,0,0,0,0
4,8537147,1,1,1,0.0,0.0,0.0,1,0.0,1,0,0.0,1,0,0,0.0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,2887903,1,1,1,0.0,0.0,0.0,1,0.0,0,0,0.0,1,1,0,0.0,1,0,1,0,0,0,0,0,0
499996,3197089,1,1,1,0.0,0.0,0.0,1,0.0,1,0,0.0,1,1,0,0.0,1,0,0,1,0,0,0,0,0
499997,2316799,1,1,1,0.0,0.0,1.0,1,0.0,0,0,0.0,1,1,1,0.0,1,0,0,1,0,0,0,0,0
499998,2319621,1,1,1,0.0,0.0,0.0,0,0.0,0,0,0.0,1,1,1,0.0,1,0,1,0,0,0,0,0,0


In [ ]:
pip install lightfm

     |████████████████████████████████| 317kB 5.1MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705346 sha256=1027939f9151aeddec187758e913621ea83fbd990b4e6c1b75f0fa1fe9245d8b
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [ ]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
import numpy as np
from IPython.display import display_html
import warnings

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
#from skopt import forest_minimize



In [ ]:
table_1_csr = csr_matrix(table_1.values)
table_2_csr = csr_matrix(table_2.values)


In [ ]:
user_trail_interaction = pd.pivot_table(table_1, index='userId', columns='trailId', values= 'event_type')


In [ ]:
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(table_2_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

In [ ]:
item_dict ={}
df = table_1[['trailId', 'event_type']].sort_values('trailId').reset_index()

for i in range(df.shape[0]):
    item_dict[(df.loc[i,'trailId'])] = df.loc[i,'trailId']

In [ ]:
userId = list(table_2.index)
user_dict = {}
counter = 0 
for i in userId:
    user_dict[i] = counter
    counter += 1

In [ ]:
def sample_recommendation_user(model, df, userId, user_dict, 
                               item_dict,threshold = 0,nrec_items = 5, show = True):
    
    n_users, n_items = df.shape
    user_x = user_dict[userId]
    scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=table_1_csr))
    scores.index = df.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(df.loc[userId,:] \
                                 [df.loc[userId,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1

In [ ]:
sample_recommendation_user(model, table_2, '57753482', user_dict, item_dict)
